Created on Aug 20, 2019

@author:     Ruchika Verma

Description: Nuclei segmentation using Mask-RCNN and based on Feature Pyramid Network (FPN) and a ResNet 50
             backbone.
            
This model was trained using data from our TMI paper and MoNuSeg challenge
  
Please cite the following paper and our MoNuSeg (https://monuseg.grand-challenge.org/) challenge if you use
this code-


N. Kumar, R. Verma, S. Sharma, S. Bhargava, A. Vahadane and A. Sethi, "A Dataset and a Technique for Generalized Nuclear Segmentation for Computational Pathology," in IEEE Transactions on Medical Imaging, 
vol. 36, no. 7, pp. 1550-1560, July 2017


In [2]:
#Process whole slide images
import os
os.add_dll_directory('/path/to/openslide/bin')
import openslide
from PIL import Image
import skimage
import imageio
import os,sys
import glob
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
from PIL import Image
import scipy
import scipy.ndimage

FileNotFoundError: Could not find module 'libopenslide-0.dll' (or one of its dependencies). Try using the full path with constructor syntax.

In [ ]:
# Read svs files from the desired path
data_path = '/media/vaid-computer/Ruchika TCGA images/TCGA_FFPES'
files=glob.glob('/media/vaid-computer/Ruchika TCGA images/TCGA_FFPES/*.svs')
len(files)

In [ ]:
# Reading a single svs file
i = 400

# Read level 3 of an svs image
image_name = files[i]
image_name
img = openslide.OpenSlide(image_name)
level = 3 #For easy visualization
image = np.array(img.read_region((0,0),level,img.level_dimensions[level]).convert('RGB'))
plt.imshow(image)
plt.show()
print(image.shape)

In [ ]:
#Extract 1000*1000 patch from a random x,y location
x,y = 1000,700
patch_size = 1000
im_patch = np.array(img.read_region((y*2**(level+2),x*2**(level+2)),0,(patch_size,patch_size)).convert('RGB')) 
plt.imshow(im_patch)

In [ ]:
# Import Mask RCNN and load my weights of my trained model
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log
import nucleus
import tensorflow as tf
config = nucleus.NucleusInferenceConfig()
config.display()
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0
with tf.device(DEVICE):
    nmodel = modellib.MaskRCNN(mode="inference",
                              model_dir=os.getcwd(),
                              config=config)
weights_path = "/media/vaid-computer/Ruchika TCGA images/Nuc_Seg/mask_rcnn_weights.h5"
nmodel.load_weights(weights_path, by_name=True)  

In [ ]:
# Run detection and predict nuclei on a patch
im_patch = skimage.exposure.equalize_adapthist(im_patch)
im_patch = skimage.exposure.adjust_log(im_patch)
im_patch = im_patch*255.0
results = nmodel.detect([im_patch], verbose=1)
r = results[0]
visualize.display_instances(im_patch, r['rois'], r['masks'],r['class_ids'],['BG', 'Nuclei'],
                      figsize=(16, 16), ax=None,
                      show_mask=False, show_bbox=False,
                      title="Predictions")  

In [ ]:
# Generate an n-ary nuclei mask 
individual_nuclei = r['masks']
predicted_nuclei = np.zeros((patch_size,patch_size), dtype = int)
n_nuc = 0
for k in range(individual_nuclei.shape[2]):
    n_nuc += 1
    nuc_mask = r['masks'][:,:,k]
    predicted_nuclei += (n_nuc)*nuc_mask
plt.imshow(predicted_nuclei)
